In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [71]:
rew=pd.read_csv('fish_train2.csv')
rew_res = pd.read_csv('fish_reserved.csv')
rew.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Roach,150.0,20.4,22.0,24.7,5.8045,3.7544
1,Perch,250.0,25.4,27.5,28.9,7.2828,4.5662
2,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
3,Bream,925.0,36.2,39.5,45.3,18.7542,6.7497
4,Perch,110.0,19.0,21.0,22.5,5.6925,3.5550


In [72]:
train_num_x_s,test_num_x_s,train_num_y,test_num_y = train_test_split(rew.drop(['Weight'],axis=1), rew['Weight'],test_size=0.2, random_state=33,stratify=rew['Species'])
# train_num_x.shape,test_num_x.shape,train_num_y.shape,test_num_y.shape

In [73]:
train_num_x = rew.drop(['Species','Weight'], axis=1)
test_num_x = rew_res.drop(['Species'], axis=1)

In [74]:
train_pca = train_num_x[['Length1','Length2','Length3']]
test_pca = test_num_x[['Length1','Length2','Length3']]

In [75]:
pca = PCA(n_components=1)
train_num_x_tr = pca.fit_transform(train_pca)
test_num_x_tr = pca.transform(test_pca)
train_num_x_tr
print(pca.explained_variance_ratio_)

[0.99627616]


In [85]:
train_num_x['Lengths'] = train_num_x_tr
test_num_x['Lengths'] = test_num_x_tr
train_x = train_num_x.drop(['Length1','Length2','Length3'],axis=1)
test_x = test_num_x.drop(['Length1','Length2','Length3'],axis=1)
# train_num_y.head()

input_pd = pd.read_csv('fish_train.csv', header=None)  
input_data = input_pd.to_numpy()
X = input_data[:, [2, 3, 4, 5, 6]]
y = input_data[:, 1]
x_0 = input_data[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33, stratify=x_0)

print(y_train)
model = LinearRegression()
model.fit(train_x, y_train)

[1000.0 540.0 840.0 161.0 130.0 145.0 700.0 340.0 120.0 685.0 900.0 430.0
 920.0 690.0 150.0 680.0 87.0 272.0 250.0 5.9 110.0 1000.0 265.0 456.0
 270.0 1000.0 19.7 925.0 9.9 720.0 242.0 600.0 60.0 700.0 10.0 115.0 430.0
 260.0 55.0 700.0 390.0 475.0 169.0 110.0 1100.0 800.0 125.0 110.0 85.0
 250.0 13.4 770.0 197.0 345.0 300.0 1550.0 820.0 850.0 140.0 685.0 8.7
 1000.0 150.0 1250.0 300.0 567.0 340.0 700.0 145.0 9.8 40.0 820.0 40.0
 270.0 500.0 0.0 450.0 160.0 363.0 200.0 150.0 955.0 300.0 188.0 1100.0
 500.0 218.0 6.7 9.8 500.0 450.0 150.0 69.0 714.0 160.0]


ValueError: Found input variables with inconsistent numbers of samples: [119, 95]

In [ ]:
for c in train_x.columns:
    train_x[c] = train_x[c]**3
for c in test_x.columns:
    test_x[c] = test_x[c]**3
train_x['Width'].mean()

inf

In [ ]:
train_x=train_x.join(rew['Species'])
test_x=test_x.join(rew_res['Species'])

In [ ]:
train_x_dum = pd.get_dummies(train_x, columns=['Species'],dtype=int)
train_x_dum.head()

,Height,Width,Lengths,Species_Bream,Species_Parkki,Species_Perch,Species_Pike,Species_Roach,Species_Smelt,Species_Whitefish
0,195.566492,52.920218,-1515.464970,0,0,0,0,1,0,0
1,386.273710,95.206103,-28.331510,0,0,1,0,0,0,0
2,1926.992424,135.321746,3.780985,1,0,0,0,0,0,0
3,6596.227555,307.505871,7738.235923,1,0,0,0,0,0,0
4,184.462936,44.928179,-2846.376115,0,0,1,0,0,0,0


In [ ]:
test_x_dum = pd.get_dummies(test_x, columns=['Species'],dtype=int)
test_x_dum.shape

KeyError: "None of [Index(['Species'], dtype='object')] are in the [columns]"

In [ ]:
reg = LinearRegression().fit(train_x_dum, rew['Weight'])
y_pred = reg.predict(test_x_dum)
y_pred.tofile('ans.txt',sep=',')